# trackingML DL with GlueX Fall 2018 data part 5

Here I am finally ready to put the pieces from the first 4 parts together to try and get a working model that trains the 5 parameters of the tracking state vector using a custom loss function that incorporates the inverse covariance matrix from the traditional tracking result. I use the "trackingML_features6.csv" and associated labels file left from the end of part 4. It only has 10k events, but should be sufficient to get a fully working system and to prove we can overtrain with the model.

The first step isto read the data into the dataframes. I print the column names again for the labels file just to confirm the format is what is expected.

In [1]:
import os
import pandas as pd
import numpy as np

TRAIN_FILE  = '/home/davidl/work2/2020.04.30.trackingML/trackingML_features6.csv'
LABELS_FILE = '/home/davidl/work2/2020.04.30.trackingML/trackingML_labels6.csv'
MAX_TRACKS  = 10  # Number of tracks(lines) to read from training file

# Get fraction of features file read so we can estimate fraction of data we're using
percent_read = 0.0;
with open(TRAIN_FILE) as f:
    f.readline()  # skip header
    for i in range(100): percent_read = percent_read + float(len(f.readline()) + 1)
    percent_read = 100.0*percent_read/os.path.getsize(TRAIN_FILE)*MAX_TRACKS/100.0

# NOTE: Only reading first few rows for now since it takes a long time to read entire file
df       = pd.read_csv(TRAIN_FILE  , nrows=MAX_TRACKS)
labelsdf = pd.read_csv(LABELS_FILE , nrows=MAX_TRACKS)

NINPUTS = len(df.columns)

print('\n\nNumber of input features per track: %d' % NINPUTS)
print('Number of tracks read: %d  ( %3.2f%% of total )' % (len(df.index), percent_read))
print('Label Names: ')
print('           ' + ', '.join(labelsdf.columns[0:1])) # event
print('           ' + ', '.join(labelsdf.columns[1:6])) # q_over_pt, phi, tanl, D, z
print('              ' + '           '*0 + ',    '.join(labelsdf.columns[6:11])) # cov_01 - cov_04
print('              ' + '           '*1 + ',    '.join(labelsdf.columns[11:15])) # cov_11 - cov_14
print('              ' + '           '*2 + ',    '.join(labelsdf.columns[15:18])) # cov_22 - cov_24
print('              ' + '           '*3 + ',    '.join(labelsdf.columns[18:20])) # cov_33 - cov_34
print('              ' + '           '*4 + ',    '.join(labelsdf.columns[20:21])) # cov_44
print('           ' + ', '.join(labelsdf.columns[21:26])) # invcov_00 - invcov_04
print('           ' + ', '.join(labelsdf.columns[26:31])) # invcov_10 - invcov_14
print('           ' + ', '.join(labelsdf.columns[31:36])) # invcov_20 - invcov_24
print('           ' + ', '.join(labelsdf.columns[36:41])) # invcov_30 - invcov_34
print('           ' + ', '.join(labelsdf.columns[41:46])) # invcov_40 - invcov_44
print('           ' + ', '.join(labelsdf.columns[46:]))   # chisq, Ndof, rms
print('\n')



Number of input features per track: 5835
Number of tracks read: 10  ( 0.10% of total )
Label Names: 
           event
           q_over_pt, phi, tanl, D, z
              cov_00,    cov_01,    cov_02,    cov_03,    cov_04
                         cov_11,    cov_12,    cov_13,    cov_14
                                    cov_22,    cov_23,    cov_24
                                               cov_33,    cov_34
                                                          cov_44
           invcov_00, invcov_01, invcov_02, invcov_03, invcov_04
           invcov_10, invcov_11, invcov_12, invcov_13, invcov_14
           invcov_20, invcov_21, invcov_22, invcov_23, invcov_24
           invcov_30, invcov_31, invcov_32, invcov_33, invcov_34
           invcov_40, invcov_41, invcov_42, invcov_43, invcov_44
           chisq, Ndof, rms




## Define global parameters

Here I define some gobal parameters used later. Gathering them here makes it a little easier to manage globally as opposed to burying them in the code. Even if it means re-running some cells when these need to change.

In [2]:
NINPUTS = len(df.columns) # Number of input values(features) per track

GPUS   = 0  # 0=force CPU, otherwise, the number of GPUs to use
Nouts  = 200 # For Lamda layers that use MyWeightedAvg()

#------------- Define output bin values and ranges
DMIN   = -12.0
DMAX   =  12.0
D_BINSIZE = (DMAX-DMIN)/Nouts

PHIMIN   = -3.5
PHIMAX   =  3.5
PHI_BINSIZE = (PHIMAX-PHIMIN)/Nouts

QOVERPt_MIN   = -2000.0
QOVERPt_MAX   =  2000.0
QOVERPt_BINSIZE = (QOVERPt_MAX-QOVERPt_MIN)/Nouts

TANLMIN   = -10.0
TANLMAX   = 400.0
TANL_BINSIZE = (TANLMAX-TANLMIN)/Nouts

ZMIN   = -100.0
ZMAX   =  400.0
Z_BINSIZE = (ZMAX-ZMIN)/Nouts
#--------------

## Model Layer Definition

This is just copied from part 2 for convenience, but the model is beefed up a bit with additional layers.

This section defines the layers of the model. It actually just defines some procedures that aren't executed until a couple of cells later. (That is when any errors will come up.)

To make things easier to maintain, different parts of the model are defined in different procedures. The structure is shown in the diagram below. Note that these make use of a Lambda layer which uses the MyWeightedAvg procedure defined at the bottom of the cell. Lamda's are nice for making simple custom layers without the complexity of full custom layers.

![ModelDiagram.png](ModelDiagram.png)

In [3]:
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Reshape, Flatten, Input, Lambda
from tensorflow.keras.optimizers import SGD, Adamax, Adadelta
from tensorflow.keras.callbacks import Callback, TensorBoard
import tensorflow.keras.backend as K
import tensorflow.keras.losses
import tensorflow as tf

#-----------------------------------------------------
# DefineCommonModel
#-----------------------------------------------------
def DefineCommonModel(inputs):
    x = Flatten(name='top_layer1')(inputs)
    x = Dense(int(Nouts*5), name='common_layer1', activation='linear', kernel_initializer="glorot_uniform")(x)
    x = Dense(int(Nouts), name='common_layer2', activation='tanh', kernel_initializer="glorot_uniform")(x)
    x = Dense(int(Nouts), name='common_layer3', activation='linear', kernel_initializer="glorot_uniform")(x)
    x = Dense(int(Nouts), name='common_layer4', activation='tanh', kernel_initializer="glorot_uniform")(x)
    x = Dense(int(Nouts), name='common_layer5', activation='linear', kernel_initializer="glorot_uniform")(x)
    return x

#-----------------------------------------------------
# DefineDModel
#-----------------------------------------------------
def DefineDModel(inputs):
    x = Dense(Nouts, name='D_output_dist1', activation='linear', kernel_initializer="glorot_uniform")(inputs)
    x = Dense(Nouts, name='D_output_dist2', activation='tanh', kernel_initializer="glorot_uniform")(x)
    x = Dense(Nouts, name='D_output_dist3', activation='linear', kernel_initializer="glorot_uniform")(x)
    x = Dense(Nouts, name='D_output_dist4', activation='relu', kernel_initializer="glorot_uniform")(x)
    x = Lambda(MyWeightedAvg, output_shape=(1,), name='D_output', arguments={'binsize':D_BINSIZE, 'xmin':DMIN})(x)
    return x

#-----------------------------------------------------
# DefinePhiModel
#-----------------------------------------------------
def DefinePhiModel(inputs):
    x = Dense(Nouts, name='phi_output_dist1', activation='linear', kernel_initializer="glorot_uniform")(inputs)
    x = Dense(Nouts, name='phi_output_dist2', activation='tanh', kernel_initializer="glorot_uniform")(x)
    x = Dense(Nouts, name='phi_output_dist3', activation='linear', kernel_initializer="glorot_uniform")(x)
    x = Dense(Nouts, name='phi_output_dist4', activation='relu', kernel_initializer="glorot_uniform")(x)
    x = Lambda(MyWeightedAvg, output_shape=(1,), name='phi_output', arguments={'binsize':PHI_BINSIZE, 'xmin':PHIMIN})(x)
    return x

#-----------------------------------------------------
# Define_q_over_pt_Model
#-----------------------------------------------------
def Define_q_over_pt_Model(inputs):
    x = Dense(Nouts, name='q_over_pt_output_dist1', activation='linear', kernel_initializer="glorot_uniform")(inputs)
    x = Dense(Nouts, name='q_over_pt_output_dist2', activation='tanh', kernel_initializer="glorot_uniform")(x)
    x = Dense(Nouts, name='q_over_pt_output_dist3', activation='linear', kernel_initializer="glorot_uniform")(x)
    x = Dense(Nouts, name='q_over_pt_output_dist4', activation='relu', kernel_initializer="glorot_uniform")(x)
    x = Lambda(MyWeightedAvg, output_shape=(1,), name='q_over_pt_output', arguments={'binsize':QOVERPt_BINSIZE, 'xmin':QOVERPt_MIN})(x)
    return x

#-----------------------------------------------------
# Define_tanl_Model
#-----------------------------------------------------
def Define_tanl_Model(inputs):
    x = Dense(Nouts, name='tanl_output_dist1', activation='linear', kernel_initializer="glorot_uniform")(inputs)
    x = Dense(Nouts, name='tanl_output_dist2', activation='tanh', kernel_initializer="glorot_uniform")(x)
    x = Dense(Nouts, name='tanl_output_dist3', activation='linear', kernel_initializer="glorot_uniform")(x)
    x = Dense(Nouts, name='tanl_output_dist4', activation='relu', kernel_initializer="glorot_uniform")(x)
    x = Lambda(MyWeightedAvg, output_shape=(1,), name='tanl_output', arguments={'binsize':TANL_BINSIZE, 'xmin':TANLMIN})(x)
    return x

#-----------------------------------------------------
# DefineZModel
#-----------------------------------------------------
def DefineZModel(inputs):
    x = Dense(Nouts, name='z_output_dist1', activation='linear', kernel_initializer="glorot_uniform")(inputs)
    x = Dense(Nouts, name='z_output_dist2', activation='tanh', kernel_initializer="glorot_uniform")(inputs)
    x = Dense(Nouts, name='z_output_dist3', activation='linear', kernel_initializer="glorot_uniform")(inputs)
    x = Dense(Nouts, name='z_output_dist4', activation='relu', kernel_initializer="glorot_uniform")(inputs)
    x = Lambda(MyWeightedAvg, output_shape=(1,), name='z_output', arguments={'binsize':Z_BINSIZE, 'xmin':ZMIN})(x)
    return x

#-----------------------------------------------------
# DefineCommonOutput
#-----------------------------------------------------
def DefineCommonOutput(inputs):
    x = tf.keras.layers.concatenate( inputs )
    x = Dense(Nouts, name='common_out1', activation='linear', kernel_initializer="glorot_uniform")(x)
    x = Dense(Nouts, name='common_out2', activation='tanh', kernel_initializer="glorot_uniform")(x)
    x = Dense(Nouts, name='common_out3', activation='relu', kernel_initializer="glorot_uniform")(x)
    x = Dense(5, name='outputs', activation='relu', kernel_initializer="glorot_uniform")(x)
    return x

#-----------------------------------------------------
# MyWeightedAvg
#
# This is used by the final Lambda layer in each branch
# of the network. It defines the formula for calculating
# the weighted average of the inputs from the previous
# layer.
#-----------------------------------------------------
def MyWeightedAvg(inputs, binsize, xmin):
    ones = K.ones_like(inputs[0,:])                       # [1, 1, 1, 1....]   (size Nouts)
    idx  = K.cumsum(ones)                                 # [1, 2, 3, 4....]   (size Nouts)
    norm = K.sum(inputs, axis=1, keepdims=True)           # normalization of all outputs by batch. shape is 1D array of size batch (n.b. keepdims=True is critical!)
    wsum = K.sum(idx*inputs, axis=1, keepdims=True)/norm  # array of size batch with weighted avg. of mean in units of bins (n.b. keepdims=True is critical!)
    output = (binsize*(wsum-0.5)) + xmin                  # convert from bins to physical units (shape batch,1)

    print('MyWeightedAvg:')
    print('       binsize = %f' % binsize)
    print('          xmin = %f' % xmin)
    print('   input shape = %s' % str(inputs.shape))
    print('  output shape = %s' % str(output.shape))

    return output

## Define Model

The full model is put together here using the parts defined in the previous cell. It also defines weights for the trivial loss function used. The weights are used here as a way of scaling the loss for each of the state parameters by $1/\sigma_i$ to account for the different units and uncertainties. This is actually not going to be sufficient in the end where full custom loss function will be required. It at least gives a simple example that can be used for the this.

In [4]:
#-----------------------------------------------------
# DefineModel
#-----------------------------------------------------
# This is used to define the model. It is only called if no model
# file is found in the model_checkpoints directory.
def DefineModel():

    # If GPUS==0 this will force use of CPU, even if GPUs are present
    # If GPUS>1 this will force the CPU to serve as orchestrator
    # If GPUS==1 this will do nothing, allowing GPU to act as its own orchestrator
    if GPUS!=1: tf.device('/cpu:0')

    # Here we build the network model.
    # This model is made of multiple parts. The first handles the
    # inputs and identifies common features. The rest are branches with
    # each determining an output parameter from those features.
    inputs         = Input(shape=(NINPUTS,), name='image_inputs')
    commonmodel    = DefineCommonModel(inputs)
    Dmodel         = DefinePhiModel(         commonmodel )
    phimodel       = DefineDModel(           commonmodel )
    q_over_ptmodel = Define_q_over_pt_Model( commonmodel )
    tanlmodel      = Define_tanl_Model(      commonmodel )
    zmodel         = DefineZModel(           commonmodel )
    commonoutput   = DefineCommonOutput([Dmodel, phimodel, q_over_ptmodel, tanlmodel, zmodel])

    # Custom loss function requires additional inputs (inverse covariance matrix)
    # We must use the "add_loss" method to get around restrictions that prevent
    # us from just passing them in as extra labels. See:
    # https://stackoverflow.com/questions/62154660/keras-valueerror-dimensions-must-be-equal-how-to-pass-label-dependent-values
    input_true     = Input(shape=(5,), name='state_true')
    input_incov    = Input(shape=(5*5,), name='invcov')
    all_inputs     = [inputs, input_incov, input_true]
    
    model          = Model(inputs=all_inputs, outputs=commonoutput)

    # Compile the model, possibly using multiple GPUs
    opt = Adadelta(clipnorm=1.0)
    if GPUS<=1 :
        final_model = model
    else:
        final_model = multi_gpu_model( model, gpus=GPUS )

    
    final_model.add_loss(customLoss( input_true, commonoutput, input_incov ) )
    final_model.compile(loss=None, optimizer=opt, metrics=['mae', 'mse', 'accuracy'])
    #final_model.compile(loss=customLoss, optimizer=opt, metrics=['mae', 'mse', 'accuracy'])
    
    return final_model


## Define custom loss function

In [5]:
#--------------------------------------------
# Define custom loss function 
def customLoss(y_true, y_pred, invcov):

    batch_size = tf.shape(y_pred)[0]  # n.b. y_pred.shape[0] will not work for some reason in tf1
    print('y_pred shape: ' + str(y_pred.shape) )  # y_pred shape is (batch, 5)
    print('y_true shape: ' + str(y_true.shape) )  # y_true shape is (batch, 5)
    print('invcov shape: ' + str(invcov.shape) )  # inconv shape is (batch, 25)
    
    y_pred = K.reshape(y_pred, (batch_size, 5,1)) # y_pred  shape is now (batch, 5,1)
    y_true = K.reshape(y_true, (batch_size, 5,1)) # y_state shape is now (batch, 5,1)
    invcov = K.reshape(invcov, (batch_size, 5,5)) # invcov  shape is now (batch, 5,5)
    
    # n.b. we must use tf.transpose here an not K.transpose since the latter does not allow perm argument
    invcov = tf.transpose(invcov, perm=[0,2,1])     # invcov shape is now (batch, 5,5)
    
    # Difference between prediction and true state vectors
    y_diff = y_pred - y_true
    
    # n.b. use "batch_dot" and not "dot"!
    y_dot = K.batch_dot(invcov, y_diff)           # y_dot shape is (batch,5,1)
    y_dot = K.reshape(y_dot, (batch_size, 1, 5))  # y_dot shape is now (batch,1,5)
    y_loss = K.batch_dot(y_dot, y_diff)           # y_loss shape is (batch,1,1)
    y_loss = K.reshape(y_loss, (batch_size,))     # y_loss shape is now (batch)
    return y_loss

#--------------------------------------------
# Test loss function
x_test = [1.0, 2.0, 3.0, 4.0, 5.0]
y_test = [1.1, 2.1, 3.1, 4.1, 5.1]
inconv_test = np.arange(0.1, 2.6, 0.1).tolist()

loss = K.eval(customLoss(K.variable([x_test,x_test,x_test]), K.variable([y_test,y_test,y_test]), K.variable([inconv_test,inconv_test,inconv_test])))
print('loss shape: '    + str(loss.shape)    )
print(loss)

#--------------------------------------------
# Define custom loss function 
def customLoss2(y_true, y_pred):

    batch_size = tf.shape(y_pred)[0]  # n.b. y_pred.shape[0] will not work for some reason in tf1
    print('y_pred shape: ' + str(y_pred.shape) )  # y_pred shape is (batch, 5)
    print('y_true shape: ' + str(y_true.shape) )  # y_true shape is (batch, 49)
    print('y_pred type: ' + str(type(y_pred) ) )  # y_pred shape is (batch, 5)
    print('y_true type: ' + str(type(y_true) ) )  # y_true shape is (batch, 49)

    # Note that y_pred only has the 5 state vector parameters while y_true contains
    # all of the labels (event, state vector, covariance matrix, inverse cov., ...)
    # We peel off the state vector and inverse covariance here which are the parts
    # we need.
    y_state = y_true[:,1:6]                         # y_state shape is now (batch, 5)
    invcov  = y_true[:,21:46]                       # invcov  shape is now (batch, 25)
    
    y_pred  = K.reshape(y_pred,  (batch_size, 5,1)) # y_pred  shape is now (batch, 5,1)
    y_state = K.reshape(y_state, (batch_size, 5,1)) # y_state shape is now (batch, 5,1)
    invcov  = K.reshape(invcov,  (batch_size, 5,5)) # invcov  shape is now (batch, 5,5)
    
    # n.b. we must use tf.transpose here an not K.transpose since the latter does not allow perm argument
    invcov = tf.transpose(invcov, perm=[0,2,1])     # invcov shape is now (batch, 5,5)
    
    # Difference between prediction and true state vectors
    y_diff = y_pred - y_state
    
    # n.b. use "batch_dot" and not "dot"!
    y_dot = K.batch_dot(invcov, y_diff)           # y_dot shape is (batch,5,1)
    y_dot = K.reshape(y_dot, (batch_size, 1, 5))  # y_dot shape is now (batch,1,5)
    y_loss = K.batch_dot(y_dot, y_diff)           # y_loss shape is (batch,1,1)
    y_loss = K.reshape(y_loss, (batch_size,))     # y_loss shape is now (batch)
    return y_loss

#--------------------------------------------
# Test loss function
#xx = np.arange(0.1, 4.9, 0.1).tolist()
#yy = [1.0, 2.0, 3.0, 4.0, 5.0]

#loss = K.eval(customLoss(K.variable([xx,xx,xx]), K.variable([yy,yy,yy])))
#print('loss shape: '    + str(loss.shape)    )
#print(loss)


y_pred shape: (3, 5)
y_true shape: (3, 5)
invcov shape: (3, 25)
loss shape: (3,)
[0.32499945 0.32499945 0.32499945]


## Load or Build model

I have found in the past that it is often good to be able to load a model that has undergone some training and continue training it. In addition, it is useful if you can keep track of how many epochs total were used in the training. In this cell, I look for the most recent model file and load it, if found. If no model file is found, then the model is defined using the above routines.

Note that at the end of this the "epochs_loaded" parameter will be set to the number of epochs completed already for this model (0 if it is new). This number is passed into the fit call later so it displays the right thing and, in turn, passes it to the routine that saves the model periodically so that it can name the file properly.

In [6]:
# Here we want to check if a model has been saved due to previous training.
# If so, then we read it in and continue training where it left off. Otherwise,
# we define the model and start fresh. 

checkpoints_dir = '/home/davidl/work2/2020.04.30.trackingML/model_checkpoints_part5'

# Look for most recent saved epoch
epoch_loaded = -1
if os.path.exists(checkpoints_dir):
    for f in os.listdir(checkpoints_dir):
        if f.startswith('model_epoch') and f.endswith('.h5'):
            e = int(f[11:-3])
            if e > epoch_loaded:
                epoch_loaded = e
                fname = checkpoints_dir+'/model_epoch%03d.h5' % epoch_loaded

if epoch_loaded > 0:
    print('Loading model: ' + fname)
    model = load_model( fname )
else:
    print('Unable to find saved model. Will start from scratch')
    model = DefineModel()
    epoch_loaded = 0

# Print summary of model
model.summary()

Unable to find saved model. Will start from scratch
MyWeightedAvg:
       binsize = 0.035000
          xmin = -3.500000
   input shape = (None, 200)
  output shape = (None, 1)
MyWeightedAvg:
       binsize = 0.120000
          xmin = -12.000000
   input shape = (None, 200)
  output shape = (None, 1)
MyWeightedAvg:
       binsize = 20.000000
          xmin = -2000.000000
   input shape = (None, 200)
  output shape = (None, 1)
MyWeightedAvg:
       binsize = 2.050000
          xmin = -10.000000
   input shape = (None, 200)
  output shape = (None, 1)
MyWeightedAvg:
       binsize = 2.500000
          xmin = -100.000000
   input shape = (None, 200)
  output shape = (None, 1)
y_pred shape: (None, 5)
y_true shape: (None, 5)
invcov shape: (None, 25)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image_inputs (InputLayer)       [(

## Fit the model

First I define the checkpoint routine that will save the full model after every epoch so we always have the latest model saved. It also removes the previous model since the disk space usage can get quite large otherwise.

The "EPOCHS" and "BS" variables can be used to set the total number of epochs to train and the batch size respectively. Note that I only do a few epochs here since it prints a few lines for each which will blow up the output when the fit is done if a lot more are used. This at least proves that it works.


In [8]:
#-----------------------------------------------------
# class checkpointModel
#-----------------------------------------------------
# There is a bug in keras that causes an error when trying to save a model
# trained on multiple GPUs. The work around is to save the original model
# at the end of every epoch using a callback. See
#    https://github.com/keras-team/kersas/issues/8694
if not os.path.exists(checkpoints_dir): os.mkdir(checkpoints_dir)
class checkpointModel(Callback):
    def __init__(self, model):
        self.model_to_save = model
    def on_epoch_end(self, epoch, logs=None):
        myepoch = epoch_loaded + epoch +1
        fname = checkpoints_dir+'/model_epoch%03d.h5' % myepoch
        old_fname = checkpoints_dir+'/model_epoch%03d.h5' % (myepoch-1)
        if os.path.exists( old_fname ):
            print('removing old model: %s' % old_fname)
            os.remove( old_fname )
        print('saving model: %s' % fname)
        self.model_to_save.save(fname)
cbk = checkpointModel( model )

cbk.on_epoch_end(-1)

EPOCHS = 10
BS     = 100

# Inputs is actually a list of 3 inputs so we can pass in the inverse covariance
# matrix. See DefineModel procedure above.
x_inp       = df.iloc[:,:]
y_true      = labelsdf.iloc[:, 1:6]   # Peel off only 5 state vector parameters
invcov_true = labelsdf.iloc[:,21:46]  # Peel off 25 inverse covariance matrix parameters

print(type(df.iloc[:,0:1]))
print(type(labelsdf.iloc[:,0:1]))
print(type(y_true))

# Fit the model
history = model.fit(
    x = [x_inp, invcov_true, y_true],
    y = y_true,
    batch_size = BS,
    epochs=EPOCHS,
    callbacks=[cbk],
    validation_split=0.2,
    shuffle=True,
    initial_epoch = epoch_loaded,
    use_multiprocessing=False
)


saving model: /home/davidl/work2/2020.04.30.trackingML/model_checkpoints_part5/model_epoch000.h5
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 233196.3281 - mae: 15.5941 - mse: 878.3378 - accuracy: 1.0000removing old model: /home/davidl/work2/2020.04.30.trackingML/model_checkpoints_part5/model_epoch000.h5
saving model: /home/davidl/work2/2020.04.30.trackingML/model_checkpoints_part5/model_epoch001.h5
1/1 [==============================] - 0s 245ms/step - loss: 233196.3281 - mae: 15.5941 - mse: 878.3378 - accuracy: 1.0000 - val_loss: 204705.3438 - val_mae: 13.5846 - val_mse: 739.2723 - val_accuracy: 1.0000
Epoch 2/10
1/1 [==============================] - ETA: 0s - loss: 232064.2969 - mae: 15.5926 - mse: 878.3093 - accuracy: 1.0000removing old model: /home/davidl/work2/2020.04.30.trackingML/model_checkpoints_part5/model_epoch001.h5
saving model: /home/dav